# Description

In this exercise, you will need to construct a function that communicates with a locally running server.  This repeats the earlier exercise where you were asked to construct this interaction using the low level `telnetlib`.  In particular, you will connect to port 2552 on `localhost`, and to the resource path `/stream` that may be queried with a GET method.  

As in the example shown in the lesson, data will arrive relatively slowly as lines; but in the server running for this exercise, it will continue infinitely.  You wish to terminate the session from the client side after a greeting longer than 20 characters is encountered. This will take a variable number of lines, since the greetings are randomized. Moreover, your function should itself be a generator function that will basically proxy the server's response.

You will write three solutions to this exercise.  All will behave the same, but one should use `http.server`, another `urllib.request`, and the final one `requests`.

A correct implementation of the function will behave like this (but with various greeting lines of similar format):

```python
>>> for data in short_greetings_VERSION():
...     print(data)

(18, 'Zdravstvuyte Mary!')
(13, 'Hej Jennifer!')
(18, 'Zdravstvuyte Mary!')
(13, 'Namaste Mary!')
(16, 'Hujambo Michael!')
(13, 'Hoi Jennifer!')
(15, 'Nǐ hǎo Michael!')
(13, 'Halløj Nancy!')
(21, 'Shikamoo Christopher!')
```

# Setup

In [ ]:
import http
import urllib
import requests

from exercise_server import start
start()

def short_greetings_http_client():
    # http.server.somefunc(...)
    path = '/stream'
    for _ in range(10):
        greeting = "Hello David!"
        yield (len(greeting), greeting)
        
def short_greetings_urllib():
    # urllib.request.somefunc(...)
    path = '/stream'
    for _ in range(10):
        greeting = "Hello David!"
        yield (len(greeting), greeting)
        
def short_greetings_requests():
    # requests.somefunc(...)
    path = '/stream'
    for _ in range(10):
        greeting = "Hello David!"
        yield (len(greeting), greeting)

# Solution

In [ ]:
def short_greetings_http_client():
    conn = http.client.HTTPConnection('localhost', port=2552)
    conn.request("GET", "/stream")
    resp = conn.getresponse()
    greet = b''
    while True:
        b = resp.read(1)
        greet += b
        if b == b'!':
            _greet = greet.decode()
            yield (len(_greet), _greet)
            if len(_greet) > 20:
                break
            greet = b''

In [ ]:
def short_greetings_urllib():
    url = 'http://localhost:2552/stream'
    with urllib.request.urlopen(url) as f:
        greet = b''
        while True:
            b = f.read(1)
            greet += b
            if b == b'!':
                _greet = greet.decode()
                yield (len(_greet), _greet)
                if len(_greet) > 20:
                    break
                greet = b''

In [ ]:
def short_greetings_requests():
    with requests.get('http://localhost:2552/stream', stream=True) as respS:
        for line in respS.iter_lines(chunk_size=1, decode_unicode=True, delimiter='!'):
            if line:
                yield (len(line), line)
            if len(line) > 20:
                break

# Test Cases

In [ ]:
from itertools import product

def test_greeting_http_client():
    seen_long = False
    for length, phrase in short_greetings_http_client():
        assert len(phrase) == length, "Length of phrase incorrect"
        if length > 20:
            seen_long = True
        else:
            assert length <= 20 or not seen_long, "Phrase neither short nor final"
    assert seen_long, "No long greeting was encountered!"
    
test_greeting_http_client()

In [ ]:
def test_greeting_urllib():
    seen_long = False
    for length, phrase in short_greetings_urllib():
        assert len(phrase) == length, "Length of phrase incorrect"
        if length > 20:
            seen_long = True
        else:
            assert length <= 20 or not seen_long, "Phrase neither short nor final"
    assert seen_long, "No long greeting was encountered!"
    
test_greeting_urllib()

In [ ]:
def test_greeting_requests():
    seen_long = False
    for length, phrase in short_greetings_requests():
        assert len(phrase) == length, "Length of phrase incorrect"
        if length > 20:
            seen_long = True
        else:
            assert length <= 20 or not seen_long, "Phrase neither short nor final"
    assert seen_long, "No long greeting was encountered!"
    
test_greeting_requests()